In [1]:
import itertools

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Set seed
tf.random.set_seed(42)

In [4]:
# Path of all datasets
num_datasets = ["./data/NORM_BCW.csv", "./data/NORM_Ecoli.csv", "./data/NORM_Iris.csv", "./data/NORM_ISOLET.csv", "./data/NORM_SDD.csv", "./data/NORM_PBC.csv", "./data/NORM_CMSC.csv", "./data/NORM_MagicGT.csv", "./data/NORM_Wine.csv"]
cat_datasets = ["./data/OH_BalanceScale.csv", "./data/OH_CarEvaluation.csv", "./data/OH_HayesRoth.csv", "./data/OH_Chess.csv", "./data/OH_Lymphography.csv", "./data/OH_Nursery.csv", "./data/OH_SoybeanSmall.csv", "./data/OH_TicTacToe.csv"]
mix_datasets = ["./data/OH_NORM_DefaultOfCCC.csv", "./data/OH_NORM_StudentPerf.csv", "./data/OH_NORM_Adult.csv", "./data/OH_NORM_InternetAdv.csv", "./data/OH_NORM_StatlogGC.csv"]

In [5]:
def data_splitter(df, classes):
    
    # Sets
    X_trains = []
    X_tests = []
    y_trains = []
    y_tests = []
    
    
    # Iteration for each class
    for c in classes:
        df_s = df[df['output']==c]
        X_train, X_test, y_train, y_test = train_test_split(df_s.drop(columns=['output']), 
                                                            df_s['output'], test_size=0.2, random_state=42)
        X_trains.append(X_train)
        X_tests.append(X_test)
        y_trains.append(y_train)
        y_tests.append(y_test)
    
    return pd.concat(X_trains), pd.concat(X_tests), pd.concat(y_trains), pd.concat(y_tests)

In [6]:
import itertools

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def generate_models(paths_datasets):
    for path_dataset in paths_datasets:
        # Generate df
        df = pd.read_csv(path_dataset)
        
        # Define the majority class as 0 and the other classes as 1
        most_common_class = df['output'].value_counts().index[0]
        df['output'] = df['output'].apply(lambda x: 0 if x==most_common_class else 1)
        
        # Get the name of the dataset
        ds_name = path_dataset.split('.')[1].split('_')[-1]
        
        # Get the possible classes of DS
        classes = list(df['output'].unique())
        
        # Split DataFrame to train and test
        X_train, X_test, y_train, y_test = data_splitter(df, classes)
            
                
        # Make y multiclass
        y_train = pd.concat([y_train, y_train.map({0:1, 1:0})], axis=1)
        y_test = pd.concat([y_test, y_test.map({0:1, 1:0})], axis=1)

        # Save model train Data indexes
        pd.DataFrame(y_train.index).rename(columns={0: 'index'}).to_csv(f'./idxstrain/{ds_name}.csv', index=False)

        # Save test Data indexes
        pd.DataFrame(y_test.index).rename(columns={0: 'index'}).to_csv(f'./idxstest/{ds_name}.csv', index=False)


        # GridSearch Parameters
        learning_rates = [0.01, 0.001, 0.0001]
        epoch_numbers = [50, 100, 500]
        nn_sizes = []
        for i in range(1, 6):
            nnsize = int((X_train.shape[1]*2+1)*i/5)
            if nnsize not in nn_sizes:
                nn_sizes.append(nnsize)
        parameters = [learning_rates, epoch_numbers, nn_sizes]
        comb_param = list(itertools.product(*parameters))

        # best scores placeholders
        best_model = []
        best_params = []
        best_auc = 0.0

        for params in comb_param:

            lr = params[0]
            epoch = params[1]
            nn_size = params[2]


            # Create model
            model = keras.Sequential(
                [layers.Dense(nn_size, activation="relu", name="layer1"),
                 layers.Dense(2, activation="softmax", name="outputLayer"),
                ])

            # Configure optimizer
            opt = tf.keras.optimizers.RMSprop(learning_rate=lr, name='RMSprop')

            # Compile
            model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

            # Train
            model.fit(X_train, y_train, epochs=epoch, verbose=0)

            # Get Prediction for train and test set
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)


            #Get Accuracies
            train_acc = accuracy_score(np.apply_along_axis(np.argmax, 1, y_train), np.apply_along_axis(np.argmax, 1, y_train_pred))
            test_acc = accuracy_score(np.apply_along_axis(np.argmax, 1, y_test), np.apply_along_axis(np.argmax, 1, y_test_pred))

            print(f"\n\nModel for {ds_name}:\nTrain Accuracy:{train_acc}\nTest Accuracy:{test_acc}\nClass Balance={y_train.sum()/y_train.shape[0]}\n\n")


            # Calculate auc score for Train Test
            fpr_train, tpr_train, thresholds_train = roc_curve(y_train.iloc[:,1:], pd.DataFrame(y_train_pred)[1])
            fpr, tpr, thresholds = roc_curve(y_test.iloc[:,1:], pd.DataFrame(y_test_pred)[1])
            
            auc_m_train = auc(fpr_train, tpr_train)
            auc_m = auc(fpr, tpr)
            
            # Report auc for the params
            with open('./all_params.txt', 'a') as f:
                f.write(f'{ds_name} {auc_m} {params} \n')

            if auc_m > best_auc and auc_m_train > 0:
                best_auc = auc_m
                best_model = model
                best_params = params
                
            # Release GPU memory
            tf.keras.backend.clear_session()
            del model

        # Report the best params and auc score for the dataset and class
        with open('./best_params.txt', 'a') as f:
            f.write(f'{ds_name} {best_auc} {best_params} \n')

        # Save
        best_model.save("./models/"+ds_name+".h5")

In [7]:
# Create models for numerical datasets
generate_models(num_datasets)
# Create models for categorical datasets
generate_models(cat_datasets)
# Create models for mixed datasets
generate_models(mix_datasets)



Model for BCW:
Train Accuracy:0.9936305732484076
Test Accuracy:0.926829268292683
Class Balance=output    0.235669
output    0.764331
dtype: float64




Model for BCW:
Train Accuracy:1.0
Test Accuracy:0.926829268292683
Class Balance=output    0.235669
output    0.764331
dtype: float64




Model for BCW:
Train Accuracy:1.0
Test Accuracy:0.9024390243902439
Class Balance=output    0.235669
output    0.764331
dtype: float64




Model for BCW:
Train Accuracy:1.0
Test Accuracy:0.926829268292683
Class Balance=output    0.235669
output    0.764331
dtype: float64




Model for BCW:
Train Accuracy:0.9426751592356688
Test Accuracy:0.9024390243902439
Class Balance=output    0.235669
output    0.764331
dtype: float64




Model for BCW:
Train Accuracy:1.0
Test Accuracy:0.8780487804878049
Class Balance=output    0.235669
output    0.764331
dtype: float64




Model for BCW:
Train Accuracy:1.0
Test Accuracy:0.8536585365853658
Class Balance=output    0.235669
output    0.764331
dtype: float64




Model



Model for Ecoli:
Train Accuracy:0.9813432835820896
Test Accuracy:0.9852941176470589
Class Balance=output    0.574627
output    0.425373
dtype: float64




Model for Ecoli:
Train Accuracy:0.9925373134328358
Test Accuracy:0.9705882352941176
Class Balance=output    0.574627
output    0.425373
dtype: float64




Model for Ecoli:
Train Accuracy:0.9813432835820896
Test Accuracy:0.9852941176470589
Class Balance=output    0.574627
output    0.425373
dtype: float64




Model for Ecoli:
Train Accuracy:0.996268656716418
Test Accuracy:0.9852941176470589
Class Balance=output    0.574627
output    0.425373
dtype: float64




Model for Ecoli:
Train Accuracy:0.9328358208955224
Test Accuracy:0.9852941176470589
Class Balance=output    0.574627
output    0.425373
dtype: float64




Model for Ecoli:
Train Accuracy:0.9477611940298507
Test Accuracy:0.9705882352941176
Class Balance=output    0.574627
output    0.425373
dtype: float64




Model for Ecoli:
Train Accuracy:0.9514925373134329
Test Accuracy:0.98



Model for Iris:
Train Accuracy:0.875
Test Accuracy:0.8333333333333334
Class Balance=output    0.666667
output    0.333333
dtype: float64




Model for Iris:
Train Accuracy:0.9083333333333333
Test Accuracy:0.8666666666666667
Class Balance=output    0.666667
output    0.333333
dtype: float64




Model for Iris:
Train Accuracy:0.9916666666666667
Test Accuracy:0.8666666666666667
Class Balance=output    0.666667
output    0.333333
dtype: float64




Model for Iris:
Train Accuracy:0.9916666666666667
Test Accuracy:0.9
Class Balance=output    0.666667
output    0.333333
dtype: float64




Model for Iris:
Train Accuracy:0.9916666666666667
Test Accuracy:0.9
Class Balance=output    0.666667
output    0.333333
dtype: float64




Model for Iris:
Train Accuracy:0.9916666666666667
Test Accuracy:0.9
Class Balance=output    0.666667
output    0.333333
dtype: float64




Model for Iris:
Train Accuracy:1.0
Test Accuracy:0.8666666666666667
Class Balance=output    0.666667
output    0.333333
dtype: float



Model for ISOLET:
Train Accuracy:1.0
Test Accuracy:0.9993589743589744
Class Balance=output    0.96152
output    0.03848
dtype: float64




Model for ISOLET:
Train Accuracy:1.0
Test Accuracy:0.9993589743589744
Class Balance=output    0.96152
output    0.03848
dtype: float64




Model for ISOLET:
Train Accuracy:1.0
Test Accuracy:0.9993589743589744
Class Balance=output    0.96152
output    0.03848
dtype: float64




Model for ISOLET:
Train Accuracy:1.0
Test Accuracy:0.9993589743589744
Class Balance=output    0.96152
output    0.03848
dtype: float64




Model for ISOLET:
Train Accuracy:1.0
Test Accuracy:0.9993589743589744
Class Balance=output    0.96152
output    0.03848
dtype: float64




Model for ISOLET:
Train Accuracy:1.0
Test Accuracy:1.0
Class Balance=output    0.96152
output    0.03848
dtype: float64




Model for ISOLET:
Train Accuracy:1.0
Test Accuracy:0.9993589743589744
Class Balance=output    0.96152
output    0.03848
dtype: float64




Model for ISOLET:
Train Accuracy:1.0
Tes



Model for PBC:
Train Accuracy:0.9748743718592965
Test Accuracy:0.9643835616438357
Class Balance=output    0.10233
output    0.89767
dtype: float64




Model for PBC:
Train Accuracy:0.9719049794426678
Test Accuracy:0.965296803652968
Class Balance=output    0.10233
output    0.89767
dtype: float64




Model for PBC:
Train Accuracy:0.9778437642759251
Test Accuracy:0.9707762557077626
Class Balance=output    0.10233
output    0.89767
dtype: float64




Model for PBC:
Train Accuracy:0.9787574234810416
Test Accuracy:0.971689497716895
Class Balance=output    0.10233
output    0.89767
dtype: float64




Model for PBC:
Train Accuracy:0.9773869346733668
Test Accuracy:0.9680365296803652
Class Balance=output    0.10233
output    0.89767
dtype: float64




Model for PBC:
Train Accuracy:0.9787574234810416
Test Accuracy:0.9671232876712329
Class Balance=output    0.10233
output    0.89767
dtype: float64




Model for PBC:
Train Accuracy:0.9741891274554592
Test Accuracy:0.9662100456621004
Class Balanc

...



Model for TicTacToe:
Train Accuracy:0.7856209150326797
Test Accuracy:0.8031088082901554
Class Balance=output    0.346405
output    0.653595
dtype: float64




Model for TicTacToe:
Train Accuracy:0.8235294117647058
Test Accuracy:0.7875647668393783
Class Balance=output    0.346405
output    0.653595
dtype: float64




Model for TicTacToe:
Train Accuracy:0.9633986928104575
Test Accuracy:0.917098445595855
Class Balance=output    0.346405
output    0.653595
dtype: float64




Model for TicTacToe:
Train Accuracy:0.984313725490196
Test Accuracy:0.9740932642487047
Class Balance=output    0.346405
output    0.653595
dtype: float64




Model for TicTacToe:
Train Accuracy:0.9882352941176471
Test Accuracy:0.9533678756476683
Class Balance=output    0.346405
output    0.653595
dtype: float64




Model for TicTacToe:
Train Accuracy:0.9934640522875817
Test Accuracy:0.9792746113989638
Class Balance=output    0.346405
output    0.653595
dtype: float64




Model for TicTacToe:
Train Accuracy:0.99607843



Model for DefaultOfCCC:
Train Accuracy:0.8412433851410476
Test Accuracy:0.8098650224962506
Class Balance=output    0.221176
output    0.778824
dtype: float64




Model for StudentPerf:
Train Accuracy:1.0
Test Accuracy:0.6870229007633588
Class Balance=output    0.46332
output    0.53668
dtype: float64




Model for StudentPerf:
Train Accuracy:1.0
Test Accuracy:0.6793893129770993
Class Balance=output    0.46332
output    0.53668
dtype: float64




Model for StudentPerf:
Train Accuracy:1.0
Test Accuracy:0.7099236641221374
Class Balance=output    0.46332
output    0.53668
dtype: float64




Model for StudentPerf:
Train Accuracy:1.0
Test Accuracy:0.732824427480916
Class Balance=output    0.46332
output    0.53668
dtype: float64




Model for StudentPerf:
Train Accuracy:1.0
Test Accuracy:0.7404580152671756
Class Balance=output    0.46332
output    0.53668
dtype: float64




Model for StudentPerf:
Train Accuracy:1.0
Test Accuracy:0.732824427480916
Class Balance=output    0.46332
output    0



Model for Adult:
Train Accuracy:0.8803363022113022
Test Accuracy:0.8426224474128666
Class Balance=output    0.240786
output    0.759214
dtype: float64




Model for Adult:
Train Accuracy:0.8846744471744472
Test Accuracy:0.8438507600184247
Class Balance=output    0.240786
output    0.759214
dtype: float64




Model for Adult:
Train Accuracy:0.8674754299754299
Test Accuracy:0.8467680024566252
Class Balance=output    0.240786
output    0.759214
dtype: float64




Model for Adult:
Train Accuracy:0.8806818181818182
Test Accuracy:0.8415476738830032
Class Balance=output    0.240786
output    0.759214
dtype: float64




Model for Adult:
Train Accuracy:0.8854038697788698
Test Accuracy:0.8421618301857823
Class Balance=output    0.240786
output    0.759214
dtype: float64




Model for Adult:
Train Accuracy:0.8919686732186732
Test Accuracy:0.8314140948871488
Class Balance=output    0.240786
output    0.759214
dtype: float64




Model for Adult:
Train Accuracy:0.8990709459459459
Test Accuracy:0.8



Model for InternetAdv:
Train Accuracy:0.9984093319194062
Test Accuracy:0.9640591966173362
Class Balance=output    0.161188
output    0.838812
dtype: float64




Model for InternetAdv:
Train Accuracy:0.9984093319194062
Test Accuracy:0.9661733615221987
Class Balance=output    0.161188
output    0.838812
dtype: float64




Model for InternetAdv:
Train Accuracy:0.9984093319194062
Test Accuracy:0.9661733615221987
Class Balance=output    0.161188
output    0.838812
dtype: float64




Model for InternetAdv:
Train Accuracy:0.9984093319194062
Test Accuracy:0.9661733615221987
Class Balance=output    0.161188
output    0.838812
dtype: float64




Model for InternetAdv:
Train Accuracy:0.9984093319194062
Test Accuracy:0.9640591966173362
Class Balance=output    0.161188
output    0.838812
dtype: float64




Model for InternetAdv:
Train Accuracy:0.9984093319194062
Test Accuracy:0.9619450317124736
Class Balance=output    0.161188
output    0.838812
dtype: float64




Model for InternetAdv:
Train Acc



Model for StatlogGC:
Train Accuracy:0.765
Test Accuracy:0.78
Class Balance=output    0.3
output    0.7
dtype: float64




Model for StatlogGC:
Train Accuracy:0.79125
Test Accuracy:0.81
Class Balance=output    0.3
output    0.7
dtype: float64




Model for StatlogGC:
Train Accuracy:0.78875
Test Accuracy:0.805
Class Balance=output    0.3
output    0.7
dtype: float64




Model for StatlogGC:
Train Accuracy:0.8
Test Accuracy:0.81
Class Balance=output    0.3
output    0.7
dtype: float64




Model for StatlogGC:
Train Accuracy:0.79875
Test Accuracy:0.78
Class Balance=output    0.3
output    0.7
dtype: float64




Model for StatlogGC:
Train Accuracy:0.79
Test Accuracy:0.795
Class Balance=output    0.3
output    0.7
dtype: float64




Model for StatlogGC:
Train Accuracy:0.8025
Test Accuracy:0.77
Class Balance=output    0.3
output    0.7
dtype: float64




Model for StatlogGC:
Train Accuracy:0.81125
Test Accuracy:0.805
Class Balance=output    0.3
output    0.7
dtype: float64




Model for Sta